In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import imutils
import numpy as np
from skimage.feature import hog
import pdb
import pickle

In [ ]:
filename = 'knn_k_25_mouse_classification.sav'
knn = pickle.load(open(filename, 'rb'))
all_datalist = os.listdir('./dataset/have_2/')
mouth_cascade = cv2.CascadeClassifier('./src/mouth.xml')
fix_size_w = 150
fix_size_h = 75
dim = (fix_size_w, fix_size_h)
avg_f_ii = np.load('avg_f.npy')

In [ ]:
ds_factor = 0.4
true_label_test = []
stgm_test = []
for idx in range(len(all_datalist)):
    get_data = os.listdir('./dataset/have_2/'+all_datalist[idx]+'/test')
    true_name =   os.listdir('./dataset/have_2/'+all_datalist[idx])[2].split('__')[1].split('.')[0]
    for i in range(len(get_data)):
        cap = cv2.VideoCapture('./dataset/have_2/'+all_datalist[idx]+'/test/'+get_data[i])
        haveFrame,im = cap.read()
        j = 0
        avg_f_i = np.zeros((fix_size_h,fix_size_w))  
        stgm_i = []
        true_label_i = []
        fame_count = 15
        unfame = True
        while(cap.isOpened()):
            haveFrame,im = cap.read()
            if (not haveFrame) or (cv2.waitKey(1) & 0xFF == ord('q') or j > 10):
                break
            if(unfame):
                print('join function')
                im = np.rot90(im)
                im = np.flipud(im)
                im = cv2.resize(im, None, fx=ds_factor, fy=ds_factor, interpolation=cv2.INTER_AREA)
                gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
                mouth_rects = mouth_cascade.detectMultiScale(gray, 1.7, 11)
                for (x,y,w,h) in mouth_rects:
                    y = int(y - 0.15*h)
                    im = im[y+10:y+10+w,x:x+h]
                    if(h > 100 and y > 400):
                        im = im[y:y+w,x:x+h]
                        break
#                 print(len(im))
                if(50 > len(im) or len(im) > 200):
                    continue
                im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)
                im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)    
                Dif_f = im-avg_f_ii
                Blur_f = cv2.GaussianBlur(im,(3,3),cv2.BORDER_DEFAULT,1)
                Fi_f = Dif_f / Blur_f
                fd, hog_image = hog(Fi_f, orientations=16, pixels_per_cell=(2,2),
                                cells_per_block=(2,2),visualize=True)
#                 cv2.imshow('im_dif', Dif_f)
#                 cv2.moveWindow('im_dif', 800, 50)
#                 cv2.imshow('im_blur', Blur_f)
#                 cv2.moveWindow('im_blur', 1150,50)
#                 cv2.imshow('Fi_f', Fi_f)
#                 cv2.moveWindow('Fi_f', 450,50)
#                 cv2.imshow('Hog_img', hog_image)
#                 cv2.moveWindow('Hog_img', 100,50)
# #                 cv2.imwrite('./output/'+str(idx)+'_'+str(i)+"_"+str(j)+".jpg",Blur_f*255)
#                 print("pp : "+str(true_name)+" Video : "+str(i)+" Frame : "+str(j))
                stgm_test.append(fd)
                true_label_test.append(true_name)
                j = j+1
                unfame = False
                fame_count = 15
            else:
                print('|',end="")
                fame_count = fame_count - 1
                if(fame_count == 0):
                    unfame = True
                    print("")
        print('Get hog for 1 video')
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
y_pred = knn.predict(stgm_test)
accuracy_score(true_label_test, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(y_true, y_pred,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
plot_confusion_matrix(true_label_test, y_pred,
                      title='Face Classificaion ')